In [25]:
import pandas as pd
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.chat_models import zhipuai
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

import numpy as np

In [57]:
# step1：加载表格数据
data = {
    '产品名称': ['变通', '益生菌', '大豆肽'],
    '成分': ['成分A1', '成分B1', '成分C1'],
    '功能': ['功能A', '功能B', '功能C'],
    '价格': [100, 200, 300]
}
df = pd.DataFrame(data)
print(df)

  产品名称    成分   功能   价格
0   变通  成分A1  功能A  100
1  益生菌  成分B1  功能B  200
2  大豆肽  成分C1  功能C  300


In [102]:
data = {
    "分类": ["变通", "益生菌"],
    "问题": ["吃了拍不吃不拍", "吃了肚子难受"],
    "答案": ["坚持吃", "少吃点"]
}
df2 = pd.DataFrame(data)
print(df2)

    分类       问题   答案
0   变通  吃了拍不吃不拍  坚持吃
1  益生菌   吃了肚子难受  少吃点


In [16]:
# step2：向量化产品名称
embeddings = HuggingFaceEmbeddings(model_name='../../../../models/m3e-base/')

In [93]:
# !pip install faiss-cpu

from langchain.docstore.document import Document

# 提取产品名称并获取向量
product_names = df['产品名称'].tolist()
# product_name_embeddings = embeddings.embed_documents(product_names)

# 创建FAISS索引
# vectorstore = FAISS.from_texts(product_names, embeddings)

# 创建带有元数据的文档列表
documents = [
    Document(page_content=name, metadata={
        'source': '产品',
        '产品名称': row['产品名称'],
        '成分': row['成分'],
        '功能': row['功能'],
        '价格': row['价格']
    })
    for name, row in zip(product_names, df.to_dict(orient='records'))
]

# 创建FAISS索引
vectorstore = FAISS.from_documents(documents, embeddings)

# 保存
vectorstore.save_local("faiss_index")



In [103]:
# !pip install faiss-cpu

from langchain.docstore.document import Document

# 提取产品名称并获取向量
question_names = df2['问题'].tolist()
category_names = df2['分类'].tolist()
# question_name_embeddings = embeddings.embed_documents(question_names)

# 创建FAISS索引
# vectorstore = FAISS.from_texts(product_names, embeddings)

# 创建带有元数据的文档列表
documents = [
    Document(page_content=f"{row['分类']} {row['问题']}", metadata={
        'source': 'qa',
        '问题': row['问题'],
        '答案': row['答案']
    })
    for name, row in zip(question_names, df2.to_dict(orient='records'))
]

# 创建FAISS索引
vectorstore = FAISS.from_documents(documents, embeddings)

# 保存
vectorstore.save_local("faiss_index_qa")



In [104]:
# 加载
vectorstore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

vectorstore_qa = FAISS.load_local("faiss_index_qa", embeddings, allow_dangerous_deserialization=True)

# print(vector_store.docstore)

print(f"vectorstore索引中的文档数量: {len(vectorstore.docstore._dict)}")
for doc_id, doc in list(vectorstore.docstore._dict.items())[:5]:
    print(f"文档ID: {doc_id}")
    print(f"文档内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")
    print()
    
# print(vectorstore_qa.docstore)

print(f"vectorstore_qa索引中的文档数量: {len(vectorstore_qa.docstore._dict)}")
for doc_id, doc in list(vectorstore_qa.docstore._dict.items())[:5]:
    print(f"文档ID: {doc_id}")
    print(f"文档内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")
    print()
    
    
vectorstore.merge_from(vectorstore_qa)
print(f"索引中的文档数量: {len(vectorstore.docstore._dict)}")
for doc_id, doc in list(vectorstore.docstore._dict.items())[:5]:
    print(f"文档ID: {doc_id}")
    print(f"文档内容: {doc.page_content}")
    print(f"元数据: {doc.metadata}")
    print()


vectorstore索引中的文档数量: 3
文档ID: a9ca09ab-da0c-4b95-b63d-cba4df6a6708
文档内容: 变通
元数据: {'source': '产品', '产品名称': '变通', '成分': '成分A1', '功能': '功能A', '价格': 100}

文档ID: 01d18a81-3964-4796-a22e-1abe409ac96a
文档内容: 益生菌
元数据: {'source': '产品', '产品名称': '益生菌', '成分': '成分B1', '功能': '功能B', '价格': 200}

文档ID: 77411963-60ab-4f7f-85e6-d9d8eb7330a6
文档内容: 大豆肽
元数据: {'source': '产品', '产品名称': '大豆肽', '成分': '成分C1', '功能': '功能C', '价格': 300}

vectorstore_qa索引中的文档数量: 2
文档ID: 5490f41d-6504-495d-9301-06e062f1c0e5
文档内容: 变通 吃了拍不吃不拍
元数据: {'source': 'qa', '问题': '吃了拍不吃不拍', '答案': '坚持吃'}

文档ID: a2764ad3-5a44-47b6-a641-b576c896545e
文档内容: 益生菌 吃了肚子难受
元数据: {'source': 'qa', '问题': '吃了肚子难受', '答案': '少吃点'}

索引中的文档数量: 5
文档ID: a9ca09ab-da0c-4b95-b63d-cba4df6a6708
文档内容: 变通
元数据: {'source': '产品', '产品名称': '变通', '成分': '成分A1', '功能': '功能A', '价格': 100}

文档ID: 01d18a81-3964-4796-a22e-1abe409ac96a
文档内容: 益生菌
元数据: {'source': '产品', '产品名称': '益生菌', '成分': '成分B1', '功能': '功能B', '价格': 200}

文档ID: 77411963-60ab-4f7f-85e6-d9d8eb7330a6
文档内容: 大豆肽
元数据: {'source': '产品'

In [106]:


# 重排模型（示例：简单排序）
def rerank_products(products, query):
    # 这里可以使用更复杂的重排模型，例如基于TF-IDF或BERT的重排
    return products


# 检索函数
def search_similar_products(query, top_k=2):
    try:
        # 搜索最相似的产品名称
        docs = vectorstore.similarity_search(query, k=top_k)
        print(docs)
        
        # 提取元数据
        similar_products = [doc.metadata for doc in docs]        
        return similar_products
    
    except Exception as e:
        raise Exception(str(e))

# 示例查询
if __name__ == "__main__":
    query = "益生菌 吃了你家产品后就拉肚子"
    top_k = 1
    similar_products = search_similar_products(query, top_k)

    print("\n最相似的产品/问答信息:")
    for product in similar_products:
        if product['source'] == '产品':
            print(f"产品名称: {product['产品名称']}")
            print(f"成分: {product['成分']}")
            print(f"功能: {product['功能']}")
            print(f"价格: {product['价格']}")
            print()
        elif product['source'] == 'qa':
            print(f"问题: {product['问题']}")
            print(f"答案: {product['答案']}")

[Document(metadata={'source': 'qa', '问题': '吃了肚子难受', '答案': '少吃点'}, page_content='益生菌 吃了肚子难受')]

最相似的产品/问答信息:
问题: 吃了肚子难受
答案: 少吃点
